In [1]:
from arcgis import GIS
try:
    gis = GIS('proa')
except:
    gis = GIS()

In [2]:
#Search for a dataset of HUCs 
search_results = gis.content.search('Watershed Boundaries owner:Federal_User_Community',
                                    outside_org=True)
search_results

[<Item title:"Watershed Boundaries" type:Map Image Layer owner:Federal_User_Community>,
 <Item title:"Great Smoky Mountains Watersheds" type:Web Map owner:Federal_User_Community>,
 <Item title:"Hydrography Viewer" type:Web Mapping Application owner:Federal_User_Community>,
 <Item title:"Great Smoky Mountains Downstream Flow Path" type:Web Map owner:Federal_User_Community>,
 <Item title:"Great Smoky Mountains Watersheds" type:Feature Layer Collection owner:Federal_User_Community>]

In [3]:
item = search_results[0]
item

<Item title:"Watershed Boundaries" type:Map Image Layer owner:Federal_User_Community>

In [4]:
item.id

'4c08f2e2b13741da96ad4a8f6aa5e36a'

In [5]:
layers = item.layers
for layer in layers:
    print (layer.properties.name, layer.properties.type)

WBDLine Feature Layer
2-digit HU (Region) Feature Layer
4-digit HU (Subregion) Feature Layer
6-digit HU (Basin) Feature Layer
8-digit HU  (Subbasin) Feature Layer
10-digit HU (Watershed) Feature Layer
12-digit HU (Subwatershed) Feature Layer
14-digit HU Feature Layer


Layers: https://developers.arcgis.com/python/guide/working-with-feature-layers-and-features/

In [6]:
huc8 = layers[4]
for fld in huc8.properties.fields:
    print (fld.name)

OBJECTID
SHAPE
TNMID
METASOURCEID
SOURCEDATADESC
SOURCEORIGINATOR
SOURCEFEATUREID
LOADDATE
GNIS_ID
AREAACRES
AREASQKM
STATES
HUC8
NAME
SHAPE_Length
SHAPE_Area


In [7]:
#from arcgis.features import FeatureLayer
print(huc8.url)

https://hydro.nationalmap.gov/arcgis/rest/services/wbd/MapServer/4


https://developers.arcgis.com/labs/rest/query-a-feature-layer/

https://hydro.nationalmap.gov/arcgis/rest/services/wbd/MapServer/4/query?where=1%3D1&text=&objectIds=&time=&geometry=%7B%0D%0A%22x%22+%3A+-78.15%2C+%22y%22+%3A+36.00%2C+%22spatialReference%22+%3A+%7B%22wkid%22+%3A+4326%7D%0D%0A%7D&geometryType=esriGeometryPoint&inSR=4326&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&having=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentOnly=false&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=html

In [8]:
strRest = 'https://hydro.nationalmap.gov/arcgis/rest/services/wbd/MapServer/4/query?where=1%3D1&text=&objectIds=&time=&geometry=%7B%0D%0A%22x%22+%3A+-78.15%2C+%22y%22+%3A+36.00%2C+%22spatialReference%22+%3A+%7B%22wkid%22+%3A+4326%7D%0D%0A%7D&geometryType=esriGeometryPoint&inSR=4326&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&having=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=&resultRecordCount=&queryByDistance=&returnExtentOnly=false&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=html'

In [9]:
strItems = strRest.split("&")
strDict = {}
for item in strItems[1:]:
    k,v = item.split('=')
    strDict[k]=v

In [10]:
strDict.keys()

dict_keys(['text', 'objectIds', 'time', 'geometry', 'geometryType', 'inSR', 'spatialRel', 'relationParam', 'outFields', 'returnGeometry', 'returnTrueCurves', 'maxAllowableOffset', 'geometryPrecision', 'outSR', 'having', 'returnIdsOnly', 'returnCountOnly', 'orderByFields', 'groupByFieldsForStatistics', 'outStatistics', 'returnZ', 'returnM', 'gdbVersion', 'historicMoment', 'returnDistinctValues', 'resultOffset', 'resultRecordCount', 'queryByDistance', 'returnExtentOnly', 'datumTransformation', 'parameterValues', 'rangeValues', 'quantizationParameters', 'featureEncoding', 'f'])

In [11]:
strDict['geometry']

'%7B%0D%0A%22x%22+%3A+-78.15%2C+%22y%22+%3A+36.00%2C+%22spatialReference%22+%3A+%7B%22wkid%22+%3A+4326%7D%0D%0A%7D'

Create the point used to select the HUC8

https://developers.arcgis.com/python/api-reference/arcgis.geometry.html?highlight=geometry#module-arcgis.geometry

In [12]:

from arcgis.geometry import Point, filters
pt = Point({"x" : -78.15, "y" : 36.00, "spatialReference" : {"wkid" : 4326}})


In [13]:
huc8.properties.capabilities

'Map,Query,Data'

In [14]:
query_result = huc8.query(where='1=1',geometry_filter=filters.intersects(pt))

In [15]:
query_result.spatial_reference

{'wkid': 102100, 'latestWkid': 3857}

In [16]:
query_result.sdf.shape

(1, 16)

In [17]:
sdf = query_result.sdf

In [19]:
sdf.HUC8

0    03020101
Name: HUC8, dtype: object